##Required Library Import

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline
import seaborn as sns
print("libraries imported successfully")

libraries imported successfully


In [3]:
datatype={'fare_amount':'float64','pickup_datetime':'object', 'pickup_longitude':'float64', 'pickup_latitude':'float64',
       'dropoff_longitude':'float64', 'dropoff_latitude':'float64', 'passenger_count':'int64'}

## loading dataset from local

In [4]:
df = pd.read_csv('EDA_data.csv',dtype = datatype,parse_dates=['pickup_datetime'])

In [5]:
df.columns

Index(['Unnamed: 0', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'year', 'month', 'day', 'weekday',
       'pickup_datetime_hour', 'distance_km'],
      dtype='object')

In [6]:
df.dtypes

Unnamed: 0                       int64
fare_amount                    float64
pickup_datetime         datetime64[ns]
pickup_longitude               float64
pickup_latitude                float64
dropoff_longitude              float64
dropoff_latitude               float64
passenger_count                  int64
year                             int64
month                            int64
day                              int64
weekday                          int64
pickup_datetime_hour             int64
distance_km                    float64
dtype: object

In [7]:
df.head(5)

,Unnamed: 0,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,weekday,pickup_datetime_hour,distance_km
0,0,13.7,2009-01-09 03:55:13,-73.970864,40.788804,-73.983761,40.743458,1,2009,1,9,4,3,5.114184
1,1,10.5,2009-11-27 05:22:25,-73.975898,40.789170,-73.982707,40.755365,1,2009,11,27,4,5,3.770195
2,2,4.1,2009-07-20 16:32:57,-74.007454,40.710194,-74.007822,40.711429,1,2009,7,20,0,16,0.139592
3,4,8.1,2009-01-21 16:09:00,-73.993962,40.735562,-74.007858,40.711645,1,2009,1,21,2,16,2.881222
4,5,5.0,2009-02-22 17:15:24,-73.973627,40.751612,-73.980555,40.741712,1,2009,2,22,6,17,1.235411


In [8]:
df.shape

(6422342, 14)

In [9]:
df.columns

Index(['Unnamed: 0', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'year', 'month', 'day', 'weekday',
       'pickup_datetime_hour', 'distance_km'],
      dtype='object')

In [10]:
X= df.drop(['fare_amount','pickup_datetime','Unnamed: 0'],axis=1)
Y= df[['fare_amount']]

In [12]:
X.columns,Y.columns

(Index(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
        'dropoff_latitude', 'passenger_count', 'year', 'month', 'day',
        'weekday', 'pickup_datetime_hour', 'distance_km'],
       dtype='object'),
 Index(['fare_amount'], dtype='object'))

##Train Test split

In [13]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size=0.3, random_state=21)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((4495639, 11), (1926703, 11), (4495639, 1), (1926703, 1))

In [11]:
X_train.dtypes

pickup_longitude        float64
pickup_latitude         float64
dropoff_longitude       float64
dropoff_latitude        float64
passenger_count           int64
year                      int64
month                     int64
day                       int64
weekday                   int64
pickup_datetime_hour      int64
distance_km             float64
dtype: object

##Model 1 - XGBOOST Regressor

In [14]:
from xgboost import XGBRegressor 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import pickle

In [15]:
xgb = XGBRegressor(random_state=21)

In [27]:
param_grid = {
'n_estimators' : [30, 50, 70],
'max_depth' :[9,12,14],
'learning_rate':[0.2,0.3]
}

In [17]:
gscv =GridSearchCV(estimator = xgb, param_grid = param_grid, cv=2, verbose=2)

In [ ]:
gscv.fit(X_train,Y_train)
gscv.best_params_

In [22]:
xgb = XGBRegressor(random_state=21,max_depth=14,n_estimators=70,learning_rate=0.3)

In [23]:
xgb.fit(X_train,Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.3, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=14, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=70, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=21,
             reg_alpha=0, reg_lambda=1, ...)

In [24]:
def eval_fun(model,X_train,Y_train,Y_test,X_test):

    Y_pred_train = model.predict(X_train)
    Y_pred_test = model.predict(X_test)
    
    r2_train = r2_score(Y_train,Y_pred_train)
    mse_train = mean_squared_error(Y_train,Y_pred_train)
    mae_train = mean_absolute_error(Y_train,Y_pred_train)
    print(f"r2_train_score ==> {r2_train} , mse_train_score ==> {mse_train} , mae_train_score ==> {mae_train}")
  
    r2_test = r2_score(Y_test,Y_pred_test)
    mse_test = mean_squared_error(Y_test,Y_pred_test)
    mae_test = mean_absolute_error(Y_test,Y_pred_test)
    print(f"r2_test_score ==> {r2_test} , mse_test_score ==> {mse_test} , mae_test_score ==> {mae_test}")

In [25]:
eval_fun(xgb,X_train,Y_train,Y_test,X_test)

r2_train_score ==> 0.8992850094416598 , mse_train_score ==> 1.847928362494491 , mae_train_score ==> 0.9060416440308677
r2_test_score ==> 0.8306207582823122 , mse_test_score ==> 3.113190471140647 , mae_test_score ==> 1.1305629783378306


In [26]:
import pickle

pickle.dump(xgb, open('xgb_model.pkl','wb'))

##Model 2 - Random Forest Regressor

In [28]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

In [16]:
param_grid_rf = {
'n_estimators' : [50, 70],
'max_depth' :[12,14],
}

In [17]:
rf = RandomForestRegressor(random_state=21)
gscv =GridSearchCV(estimator = rf, param_grid = param_grid_rf, cv=2, verbose=2)

In [18]:
gscv.fit(X_train,Y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] END ......................max_depth=12, n_estimators=50; total time=28.4min
[CV] END ......................max_depth=12, n_estimators=50; total time=54.6min
[CV] END ......................max_depth=12, n_estimators=70; total time=54.9min
[CV] END ......................max_depth=12, n_estimators=70; total time=48.3min
[CV] END ......................max_depth=14, n_estimators=50; total time=26.6min
[CV] END ......................max_depth=14, n_estimators=50; total time=23.1min
[CV] END ......................max_depth=14, n_estimators=70; total time=33.0min
[CV] END ......................max_depth=14, n_estimators=70; total time=49.7min


GridSearchCV(cv=2, estimator=RandomForestRegressor(random_state=21),
             param_grid={'max_depth': [12, 14], 'n_estimators': [50, 70]},
             verbose=2)

In [19]:
gscv.best_params_

{'max_depth': 14, 'n_estimators': 70}

In [31]:
rf1 = RandomForestRegressor(n_estimators=70,max_depth=14,oob_score=True,random_state=21,verbose = 2)

In [32]:
rf1.fit(X_train,Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   54.9s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 92.0min finished


RandomForestRegressor(max_depth=18, oob_score=True, random_state=21, verbose=2)

In [14]:
def eval_fun(model,X_train,Y_train,Y_test,X_test):

    Y_pred_train = model.predict(X_train)
    Y_pred_test = model.predict(X_test)
    
    r2_train = r2_score(Y_train,Y_pred_train)
    mse_train = mean_squared_error(Y_train,Y_pred_train)
    mae_train = mean_absolute_error(Y_train,Y_pred_train)
    print(f"r2_train_score ==> {r2_train} , mse_train_score ==> {mse_train} , mae_train_score ==> {mae_train}")
  
    r2_test = r2_score(Y_test,Y_pred_test)
    mse_test = mean_squared_error(Y_test,Y_pred_test)
    mae_test = mean_absolute_error(Y_test,Y_pred_test)
    print(f"r2_test_score ==> {r2_test} , mse_test_score ==> {mse_test} , mae_test_score ==> {mae_test}")

In [33]:
eval_fun(rf1,X_train,Y_train,Y_test,X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  3.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.5min finished


r2_train_score ==> 0.8471130880556285 , mse_train_score ==> 2.8051838089836973 , mae_train_score ==> 1.1442179833216164
r2_test_score ==> 0.8061785738499323 , mse_test_score ==> 3.562437822215551 , mae_test_score ==> 1.2623880816277293


In [34]:
import pickle

pickle.dump(rf1, open('rf_model_84_80.pkl','wb'))

#Multiple Linear Regression 

In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
LinReg =  LinearRegression()

In [13]:
LinReg.fit(X_train,Y_train)

LinearRegression()

In [17]:
eval_fun(LinReg,X_train,Y_train,Y_test,X_test)

r2_train_score ==> 0.353789936921559 , mse_train_score ==> 11.856724575675564 , mae_train_score ==> 2.3411035502605833
r2_test_score ==> 0.36233953711864586 , mse_test_score ==> 11.720199339268031 , mae_test_score ==> 2.3417042191044115
